In [26]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 상품마스터

In [6]:
start_row = re.compile('^\d+\|')

In [8]:
with open('./data/마스터+정보+및+데이터+정의서/02.설명이추가된 상품마스터_20210325.txt', 'r', encoding='utf-16') as f:
    cols = f.readline().replace('\n', '').split('|')
    rows = []
    past_line = ''
    while True:
        curr_line = f.readline()
        if not curr_line:
            break

        flag = start_row.search(curr_line)

        if not flag:
            past_line += ' ' + curr_line
        elif curr_line.count('|') == 45:
            if past_line:
                row = past_line.replace('\n', '').split('|')
                rows.append(row)
                past_line = ''
            row = curr_line.replace('\n', '').split('|')
            rows.append(row)
        else:
            past_line = curr_line

In [9]:
df_desc = pd.DataFrame(rows, columns=cols)
df_desc.shape

(41315, 46)

In [21]:
df_desc.to_csv('./data/prep/02.설명이추가된 상품마스터_20210325.csv', index=False, sep='|')

In [2]:
df_desc = pd.read_csv('./data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|')

/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df_desc.keys()

Index(['PLU_CD', '상품명', '대분류명', '중분류명', '소분류명', '카테고리코드', '상품속성코드', '상품속성명',
       '판매가능여부', '시작일자', '종료일자', '사용여부', '상온저온유형코드', '상온저온유형명', '시즌상품유형코드',
       '시즌상품유형명', '행사상품유형코드', '행사상품유형명', '특별상품유형코드', '특별상품유형명', '낱개상품코드',
       '신상품주기코드', '신상품주기명', '신상품시작일자', '신상품종료일자', '점포발주여부', '점포발주가능일자',
       '발주권장유형코드', '발주권장유형명', '반품유형코드', '반품유형명', '점포발주입수수량', '공급가', '매가',
       '점포유통기한일수', 'TRANL_CD', 'LT', 'MON_YN', 'TUE_YN', 'WED_YN', 'THU_YN',
       'FRI_YN', 'SAT_YN', 'SUN_YN', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT'],
      dtype='object')

In [15]:
df_desc.iloc[:2, :15]

,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,종료일자,사용여부,상온저온유형코드,상온저온유형명,시즌상품유형코드
0,9414453003319,KGB 크랜베리맛,자점,자점,자점,1151551055,401,일반,예,20140101,20190827,아니오,0,상온,01
1,9331275008295,레돈도)카푸치노125g,과자/빵,과자행사,비스켓행사(빼빼로),2092112004,401,일반,예,20200918,99991231,예,0,상온,01


In [16]:
df_desc.iloc[:2, 15:30]

,시즌상품유형명,행사상품유형코드,행사상품유형명,특별상품유형코드,특별상품유형명,낱개상품코드,신상품주기코드,신상품주기명,신상품시작일자,신상품종료일자,점포발주여부,점포발주가능일자,발주권장유형코드,발주권장유형명,반품유형코드
0,일반,01,일반,00,기본,,,,,,0,20140831,2,선택,0
1,일반,01,일반,00,기본,,04,4주,20200917,20201015,1,20200925,2,선택,2


In [17]:
df_desc.iloc[:2, 30:]

,반품유형명,점포발주입수수량,공급가,매가,점포유통기한일수,TRANL_CD,LT,MON_YN,TUE_YN,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT
0,불가,1,2400,3500,0,010221,1,0,0,0,0,0,0,0,,
1,한도,1,1700,2700,150,010092,1,1,1,1,1,1,1,0,1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품


In [58]:
# 마스터 데이터
masters = {}
for f in sorted(Path('./data/이마트24_마스터데이터').glob('*.txt')):
    masters[f.name[3:-13]] = pd.read_csv(f, encoding="utf-16", sep='|', low_memory=False, dtype=str)
    print(f)
    print(masters[f.name[3:-13]].shape)

data/이마트24_마스터데이터/02.상품마스터_20210305.txt
(48981, 44)
data/이마트24_마스터데이터/03.행사마스터_20210305.txt
(142741, 32)
data/이마트24_마스터데이터/04.상품가격마스터_20210305.txt
(335736, 10)


In [61]:
imgs = set([f.name[:-4] for f in Path('./data/상품이미지+파일/').glob('*')])
cds = set(masters['상품마스터']['PLU_CD'].tolist())

In [65]:
len(imgs), len(cds)

(27835, 48981)

In [64]:
len(imgs & cds)

25683

# 거래데이터

In [35]:
# 거래데이터
emart24 = Path('./data/emart24Data')
f = emart24 / '001_TRAN/IFAI_001_TRAN_00115.txt'
# f = emart24 / '007_DAYQTY/IFAI_007_DAYQTY_00115.txt'
# f = emart24 / '008_STRPUR/IFAI_008_STRPUR_00115.txt'
# f = emart24 / '009_STRORD/IFAI_009_STRORD_00115.txt'
# f = emart24 / 'IFAI_100_RCMQTY_03193.txt'
temp = pd.read_csv(f, sep='|', low_memory=False)
temp

,DT,ORG_CD,PLU_CD,GDS_NM,LCLSS_CD,LCLSS_NM,MCLSS_CD,MCLSS_NM,SCLSS_CD,SCLSS_NM,...,DMT_OUT_QTY,DISU_QTY,STRUSE_QTY,STCK_BAL_DIFF_QTY,STCK_DIFF_QTY,SHIFT_STCK_QTY,WARIN_EDT_QTY,MANU_IN_QTY,MANU_GDS_QTY,END_QTY
0,20190321,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,11.0
1,20190912,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,12.0
2,20190916,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,13.0
3,20200625,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,15.0
4,20190506,A1600115,135,"쓰레기봉투검정(90X110, 박스:100장)",114,소모품,144100,소모품,1001,청소용품,...,0,0,0,0.0,0,0,0,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548952,20190610,A1600115,9791196067694,도서)12가지 인생의법칙,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,1.0
548953,20191112,A1600115,9791196067694,도서)12가지 인생의법칙,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,0.0
548954,20190612,A1600115,9791196316808,도서)공부머리 독서법,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,1.0
548955,20200629,A1600115,9791196831059,도서)코로나투자전쟁,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,1.0


In [61]:
temp.iloc[0, :]

DT                                   20190321
ORG_CD                               A1600115
PLU_CD                                    129
GDS_NM               POS감열지(표준사양,79*70mm,50롤)
LCLSS_CD                                  114
LCLSS_NM                                  소모품
MCLSS_CD                               144100
MCLSS_NM                                  소모품
SCLSS_CD                                 1002
SCLSS_NM                                영업소모품
TAX_TP                                      0
TAX_TP_NM                                  과세
GDS_COST                              37530.0
GDS_SAL_PRC                                 0
START_QTY                                11.0
WARIN_QTY                                   0
WARIN_BAL_QTY                               0
BUY_WARIN_QTY                               0
RTN_QTY                                     0
RTN_BAL_QTY                                 0
RTN_DISU_QTY                                0
SAL_QTY                           

In [58]:
temp.keys()

Index(['DT', 'ORG_CD', 'PLU_CD', 'GDS_NM', 'LCLSS_CD', 'LCLSS_NM', 'MCLSS_CD',
       'MCLSS_NM', 'SCLSS_CD', 'SCLSS_NM', 'TAX_TP', 'TAX_TP_NM', 'GDS_COST',
       'GDS_SAL_PRC', 'START_QTY', 'WARIN_QTY', 'WARIN_BAL_QTY',
       'BUY_WARIN_QTY', 'RTN_QTY', 'RTN_BAL_QTY', 'RTN_DISU_QTY', 'SAL_QTY',
       'SAL_RTN_QTY', 'SHIFT_WARIN_QTY', 'SHIFT_OUT_QTY', 'DMT_IN_QTY',
       'DMT_OUT_QTY', 'DISU_QTY', 'STRUSE_QTY', 'STCK_BAL_DIFF_QTY',
       'STCK_DIFF_QTY', 'SHIFT_STCK_QTY', 'WARIN_EDT_QTY', 'MANU_IN_QTY',
       'MANU_GDS_QTY', 'END_QTY'],
      dtype='object')